<a href="https://colab.research.google.com/github/hemsush/chatgpt_prompts_exercises/blob/main/PromptTechniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prompt Technique exercises**

In [3]:
!pip install openai

yRequirement already satisfied: openai in /usr/local/lib/python3.10/dist-packages (1.57.4)


In [4]:
from openai import OpenAI

In [5]:
from google.colab import userdata
# Initialize the OpenAI client
client = OpenAI(
    api_key=userdata.get('open_api_key')
)

In [6]:
#install dataset
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
from datasets import load_dataset

In [8]:
# Load the MedQA dataset
dataset = load_dataset('bigbio/med_qa')

# Select a sample from the dataset for testing
sample = dataset['train'][0]


README.md:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

med_qa.py:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

bigbiohub.py:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

The repository for bigbio/med_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/med_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


data_clean.zip:   0%|          | 0.00/132M [00:00<?, ?B/s]

...

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/train.jsonl


Generating test split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/test.jsonl


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/dev.jsonl


In [9]:
# Define function to call the OpenAI API
def chat_gpt(prompt, model):
    response = client.chat.completions.create(
        model=model,  # Model selection (either gpt-4o-mini or gpt-4o)
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# Zero-Shot Prompting Method
def zero_shot_prompt(question, model):
    prompt = f"Question: {question}\nAnswer:"
    return chat_gpt(prompt, model)

# One-Shot Prompting Method
def one_shot_prompt(question, example, model):
    prompt = f"Example: {example}\nQuestion: {question}\nAnswer:"
    return chat_gpt(prompt, model)

# Few-Shot Prompting Method
def few_shot_prompt(question, examples, model):
    prompt = "\n".join([f"Q: {ex[0]} A: {ex[1]}" for ex in examples])
    prompt += f"\nQ: {question} A:"
    return chat_gpt(prompt, model)

# Define example for One-Shot and Few-Shot
example_1 = ("What is the function of insulin in the body?", "Insulin is a hormone that helps regulate blood sugar levels by facilitating the absorption of glucose into cells.")
example_2 = ("What is the function of adrenaline in the body?", "Adrenaline is a hormone that prepares the body for 'fight or flight' by increasing heart rate, blood flow to muscles, and dilating the airways.")
few_shot_examples = [
    example_1,
    example_2
]

# Question from MedQA dataset
question = sample['question']

# Model options
models = {
    "GPT-4o-mini": "gpt-4o-mini",
    "GPT-4o": "gpt-4o"
}

# Collect results for both models
results = {}

# Loop over both models to compare their performance
for model_name, model_id in models.items():
    print(f"Model: {model_name}\n")

    # Zero-Shot Output
    zero_shot_output = zero_shot_prompt(question, model_id)
    print(f"Zero-Shot Output: {zero_shot_output}\n")

    # One-Shot Output
    one_shot_output = one_shot_prompt(question, example_1, model_id)
    print(f"One-Shot Output: {one_shot_output}\n")

    # Few-Shot Output
    few_shot_output = few_shot_prompt(question, few_shot_examples, model_id)
    print(f"Few-Shot Output: {few_shot_output}\n")

    results[model_name] = {
        "Zero-Shot": zero_shot_output,
        "One-Shot": one_shot_output,
        "Few-Shot": few_shot_output
    }

# Display the results
for model_name, result in results.items():
    print(f"Summary for {model_name}:")
    for prompt_type, output in result.items():
        print(f"  {prompt_type} Output: {output}\n")

Model: GPT-4o-mini

Zero-Shot Output: The symptoms described in the vignette suggest that the patient is experiencing a urinary tract infection (UTI), which is common in pregnancy. Given that the condition has worsened despite drinking more water and taking cranberry extract, and considering the patient's physical exam is otherwise normal with no signs of systemic infection, the best treatment for this patient would be an antibiotic.

In pregnant women, the first-line antibiotics for uncomplicated UTIs include:

- **Nitrofurantoin** (but not in the third trimester due to potential risk of hemolytic anemia in the newborn),
- **Cephalexin**, or
- **Amoxicillin** (unless there is known resistance).

Among these options, **Cephalexin** is often preferred due to its safety profile and efficacy. Therefore, the best treatment for this patient would be to prescribe **Cephalexin** for her urinary tract infection.

One-Shot Output: The patient's symptoms are suggestive of a urinary tract infecti